### VERGE: Vector-mode Regional Geospatial Embeddings

# Assemble geographical data for the VERGE project

This notebook pulls and organizes the geospatial data that we will use in this effort.

We have a list of cities and towns of a reasonable size in
NH, ME, and VT.
For each one, we will pull geo entities within a 10 km box, and
thn will break that up into 1km boxes.
We will only keep cases that contain a minimum amout of stuff.

## Colab or local setup

In [ ]:
# !pip install osmnx pygeohash geo-encodings

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# project_home = '/content/drive/MyDrive/Projects/verge'

In [ ]:
# If running locally:
data_dname = '../data'

## Setup

In [ ]:
import numpy as np
import pandas as pd
import pyproj
import shapely
import osmnx
import pygeohash
import geopandas as gpd
import os
import copy

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import draw_shape

import sys
sys.path.append(project_home)
from utils.verge import rules


## Parameters

In [ ]:
# AOIs are squares of this dimension, in meters.
aoi_size = 10000

# This is the size of the square tiles that make up the AOI.
tile_size = 1000

# Successive tiles differ by this much.
tile_shift = 1000

# A tile has to contain this many entities in order to be retained.
min_entity_count = 20

# Set to True if we want to include the overall land/water polygon for each tile.
include_land_water = True


## Preliminaries

In [ ]:
# Define a local map projection

def get_projections(lon, lat):

    center_lat = lat
    center_lon = lon
    x0 = aoi_size / 2
    y0 = aoi_size / 2

    proj_def = f"""
    +proj=tmerc +lat_0={center_lat} +lon_0={center_lon}
    +k=1.0 +x_0={x0} +y_0={y0} +datum=WGS84 +units=m +no_defs
    """

    ltm_crs = pyproj.CRS.from_proj4(proj_def)
    wgs84_crs = pyproj.CRS.from_epsg(4326)
    proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
    proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

    return proj_forward, proj_inverse

In [ ]:
# Define a polygon for the AOI bounds.

def get_aoi_bbox(aoi_size):
    x0, y0 = 0, 0
    x1, y1 = aoi_size, aoi_size
    xx = [x0, x1, x1, x0, x0]
    yy = [y0, y0, y1, y1, y0]
    aoi_bbox = shapely.Polygon(list(zip(xx, yy)))
    return aoi_bbox

In [ ]:
# This function gets an overall land/water polygon for an AOI.
# It does it by conasidering both the "coastlines" shapefile
# and all polygonal water features.

# Read the coastline file.
if include_land_water:
    fname = '%s/data/nh-vt-me-coastlines' % project_home
    coastlines_gdf = gpd.read_file(fname)
    print('%d coastline polygons' % len(coastlines_gdf))

def get_land_water(bounds, features):

    # Create a baseline polygon consisting of the whole AOI.
    landwater = copy.deepcopy(bounds)

    # Intersect that with the coastlines data.
    coastlines = shapely.union_all(coastlines_gdf['geometry'].values)
    landwater = landwater.intersection(coastlines)

    # subtract out any polygonal water feature.
    for _, f in features.iterrows():
        if f['geometry'].geom_type in ['Polygon', 'MultiPolygon']:
            if f['natural'] == 'water':
                landwater = shapely.difference(landwater, f['geometry'])

    return landwater

## Processing

In [ ]:
# Read the file with the list of AOIs
fname = '%s/data/nh-vt-me-places-over-10k.csv' % project_home
aois = pd.read_csv(fname).to_dict('records')
aois = np.random.permutation(aois)

# aois = [
#     # {'name': 'Keene NH', 'lat':  42.934, 'lon': -72.278},
#     {'name': 'Exeter NH', 'lat':  42.981163, 'lon': -70.946524},
# ]

print('%d areas of interest' % len(aois))

In [ ]:
from utils.verge import rules

# This will save some extra info that we will need.
tile_info = []

for aoi in aois:

    print('\nprocessing AOI:', aoi)
    lon = aoi['Longitude']
    lat = aoi['Latitude']

    # Get a string identifier for this AOI.
    aoi_id = pygeohash.encode(latitude=lat, longitude=lon, precision=8)
    print('aoi_id', aoi_id)

    # Get forward and inverse projections.
    proj_forward, proj_inverse = get_projections(lon, lat)

    # Use that projection to define lon/lat bounds for the query below. Make sure the bounds go
    # a little farther out than necessary to avoid edge artifacts from map projections.
    buf = 100
    x0, y0 = 0, 0
    x1, y1 = aoi_size, aoi_size
    lon0, lat0 = proj_inverse(x0 - buf, y0 - buf)
    lon1, lat1 = proj_inverse(x1 + buf, y1 + buf)
    query_bounds = [lon0, lat0, lon1, lat1]

    # print('re-projected query bounds, with buffer:')
    # print(proj_forward(lon0, lat0))
    # print(proj_forward(lon1, lat1))

    # Query for all the geospatial entities we need within the bounding box.
    tags = {
        'landuse': True,
        'place': True,
        'highway': True,
        'railway': True,
        'aeroway': True,
        'bridge': True,
        'tunnel': True,
        'power': True,
        'natural': True,
        'waterway': True,
        'landcover': True,
        'building': True,
        'amenity': True,
        'shop': True,
        'leisure': True
    }
    features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
    print('%d features from OSM' % len(features))

    # Re-format and filter everything.
    # BTW, "gents" is "geospatial entities".
    gents = []
    for feature in features.to_dict('records'):

        geomxy = shapely.ops.transform(proj_forward, feature['geometry'])
        if geomxy.is_empty:
            continue
        gtype = geomxy.geom_type

        for rule in rules:
            if gtype == rule['gtype']:
                osm_key = rule['osm_key']
                if osm_key in feature:
                    osm_value = str(feature[osm_key])
                    if osm_value in rule['osm_values']:
                        gents.append({
                            'feature': feature,
                            'category': rule['gent_category'],
                            'label': rule['gent_label'],
                            'geomxy': geomxy,
                            'gtype': gtype
                        })
    print('%d features selected' % len(gents))

    # We need some special handling to create a general "land/water" polygon.
    if include_land_water:
        lons = [lon0, lon1, lon1, lon0, lon0]
        lats = [lat0, lat0, lat1, lat1, lat0]
        lonlat_bounds = shapely.Polygon(list(zip(lons, lats)))
        landwater = get_land_water(lonlat_bounds, features)
        landwaterxy = shapely.ops.transform(proj_forward, landwater)
        gents.append({
            'category': 'waterway',
            'label': 'land',
            'geomxy': landwaterxy,
            'gtype': landwaterxy.geom_type
        })

    # Loop over tiles within this AOI.
    x0 = 0
    while x0 < aoi_size:

        y0 = 0
        while y0 < aoi_size:

            # Figure out the lon/lat center of this tile. We will need it
            # for some later processing.
            xc = x0 + tile_size / 2
            yc = y0 + tile_size / 2
            tile_lon, tile_lat = proj_inverse(xc, yc)

            x1 = x0 + tile_size
            y1 = y0 + tile_size
            xx = [x0, x1, x1, x0, x0]
            yy = [y0, y0, y1, y1, y0]
            tile_bbox = shapely.Polygon(list(zip(xx, yy)))

            tile_gents = []
            for gent in gents:
                geomxy = shapely.affinity.translate(
                    gent['geomxy'].intersection(tile_bbox),
                    xoff=-x0, yoff=-y0
                )
                if geomxy.is_empty:
                    continue
                tile_gents.append({
                    'category': gent['category'],
                    'label': gent['label'],
                    'geometry': geomxy,
                    'gtype': gent['gtype'],
                    'xoff': x0,
                    'yoff': y0,
                })

            # Save that if it's big enough.
            if len(tile_gents) >= min_entity_count:
                tx = '%02d' % (x0 / tile_shift)
                ty = '%02d' % (y0 / tile_shift)
                fname = '%s/data/tiles/%s/%s-%s.pq' % (project_home, aoi_id, tx, ty)
                os.makedirs(os.path.dirname(fname), exist_ok=True)
                gdf = gpd.GeoDataFrame(tile_gents).drop_duplicates()
                gdf.to_parquet(fname, index=False, compression="zstd")
                # print('wrote %s (%d)' % (fname, len(gdf)))

                # Save the tile info.
                tile_info.append({
                    'fname': fname,
                    'center_lon': tile_lon,
                    'center_lat': tile_lat
                })

            y0 += tile_shift
        x0 += tile_shift

# Save the tile info.
fname = '%s/data/tile_info.csv' % project_home
pd.DataFrame(tile_info).to_csv(fname, index=False)
print('saved tile info in %s' % fname)


In [ ]:
# for g in tile_gents:
#     print(g['geometry'].bounds)

In [ ]:
# For debugging.

# def pr_feature(f):
#     print('feature')
#     for k in f:
#         if str(f[k]) != 'nan':
#             print('    ', k, f[k])

# def pr_gent(g):
#     for k in g:
#         if k == 'feature':
#             pr_feature(g[k])
#         else:
#             print(k, g[k])
#         geomxy = shapely.ops.transform(proj_inverse, feature['geometry'])

# pr_gent(gents[121])

In [ ]:
pd.DataFrame(gents)[['category', 'label', 'gtype']].value_counts().sort_index()

## QA Check

In [ ]:
# Take a look at the land/water polygons.
import folium
center_lon = (lon0 + lon1) / 2.0
center_lat = (lat0 + lat1) / 2.0

map_center = [center_lat, center_lon]
m = folium.Map(location=map_center, zoom_start=10)
geo_json = folium.GeoJson(landwater)
geo_json.add_to(m)
m


In [ ]:
import folium
center_lon = (lon0 + lon1) / 2.0
center_lat = (lat0 + lat1) / 2.0

map_center = [center_lat, center_lon]
m = folium.Map(location=map_center, zoom_start=10)
for gent in tile_gents:
    g0 = gent['geometry']
    g1 = shapely.affinity.translate(g0, xoff=gent['xoff'], yoff=gent['yoff'])
    g2 = shapely.ops.transform(proj_inverse, g1)
    geo_json = folium.GeoJson(g2)
    geo_json.add_to(m)
m
